<a href="https://colab.research.google.com/github/gimenopea/CachedHE/blob/main/Praxis_Benchmark_1_Baseline_Uncached_and_Cached.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!apt-get update -qq
!apt-get install -y -qq libopenmpi-dev openmpi-bin
!pip install --quiet mpi4py phe numpy tenseal



W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 466.3/466.3 kB 8.4 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.7/53.7 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.8/4.8 MB 67.8 MB/s eta 0:00:00


#1. Paillier Uncached

In [ ]:
%%writefile mpi_paillier_homomorphic_experiment_uncached.py

from mpi4py import MPI
import numpy as np
from phe import paillier
import csv
import os
import math

def encryption_experiment(comm, total_numbers, public_key):
    """
    Direct encryption: encrypt each number directly with the public key
    """
    rank = comm.Get_rank()
    size = comm.Get_size()
    # scatter the data
    data = np.arange(1, total_numbers + 1, dtype='int') if rank == 0 else None
    chunk_size = total_numbers // size
    chunk = np.empty(chunk_size, dtype='int')
    comm.Scatter(data, chunk, root=0)

    # encrypt directly without cache
    start_time = MPI.Wtime()
    local_encrypted_chunk = []
    for x in chunk:
        # Convert numpy.int64 to Python int before encryption
        cipher = public_key.encrypt(int(x))
        local_encrypted_chunk.append(cipher)
    encryption_time = MPI.Wtime() - start_time

    return encryption_time, local_encrypted_chunk

def homomorphic_addition_experiment(comm, local_encrypted_chunk, private_key):
    start_time = MPI.Wtime()
    local_sum = local_encrypted_chunk[0]
    for ct in local_encrypted_chunk[1:]:
        local_sum = local_sum + ct

    gathered = comm.gather(local_sum, root=0)
    if comm.Get_rank() == 0:
        final = gathered[0]
        for s in gathered[1:]:
            final = final + s
        decrypted = private_key.decrypt(final)
    else:
        decrypted = None

    addition_time = MPI.Wtime() - start_time
    return addition_time, decrypted

def run_experiment(exp_cores, total_numbers, public_key, private_key, world_comm, iterations=5):
    rank = world_comm.Get_rank()
    size = world_comm.Get_size()

    # split communicator for this core count
    if size == exp_cores:
        sub_comm = world_comm
    else:
        color = 0 if rank < exp_cores else MPI.UNDEFINED
        sub_comm = world_comm.Split(color, rank)
    if sub_comm == MPI.COMM_NULL:
        return None

    enc_times, add_times, decrypted_list = [], [], []

    for _ in range(iterations):
        et, enc_chunk = encryption_experiment(sub_comm, total_numbers, public_key)
        at, dec = homomorphic_addition_experiment(sub_comm, enc_chunk, private_key)
        enc_times.append(et)
        add_times.append(at)
        if sub_comm.Get_rank() == 0:
            decrypted_list.append(dec)

    if sub_comm.Get_rank() == 0:
        avg_enc = np.mean(enc_times)
        std_enc = np.std(enc_times)
        avg_add = np.mean(add_times)
        std_add = np.std(add_times)
        expected_sum = total_numbers * (total_numbers + 1) // 2
        accuracy = sum(1 for dec in decrypted_list if dec == expected_sum) / len(decrypted_list)

        print(f"Experiment with {exp_cores} cores:")
        print(f"  Encryption: avg = {avg_enc:.6f}s, std = {std_enc:.6f}s")
        print(f"  Addition:   avg = {avg_add:.6f}s, std = {std_add:.6f}s")
        print(f"  Accuracy:   {accuracy*100:.1f}% (expected sum = {expected_sum})\n")

        return {
            "cores": exp_cores,
            "avg_enc": avg_enc,
            "std_enc": std_enc,
            "avg_add": avg_add,
            "std_add": std_add,
            "accuracy": accuracy
        }
    return None

def main():
    world = MPI.COMM_WORLD
    rank = world.Get_rank()
    size = world.Get_size()

    total_numbers = 32768
    experiment_core_counts = [4, 8, 16, 32]
    key_sizes = [512, 1024]
    iterations = 10
    results = []

    if rank == 0:
        print('== Paillier Experiment Uncached ==')

    for n_bits in key_sizes:
        if rank == 0:
            print(f"\n-- Generating {n_bits}-bit key pair --")
            public_key, private_key = paillier.generate_paillier_keypair(n_length=n_bits)
        else:
            public_key = private_key = None

        public_key = world.bcast(public_key, root=0)
        private_key = world.bcast(private_key, root=0)

        for c in experiment_core_counts:
            if size >= c:
                res = run_experiment(c, total_numbers, public_key, private_key, world, iterations)
                if rank == 0 and res:
                    res["key_size"] = n_bits
                    results.append(res)
            elif rank == 0:
                print(f"Skipping {c} cores (only {size} available)")

    if rank == 0 and results:
        csv_path = "/content/drive/MyDrive/praxisfiles/Benchmark/paillier_experiment_uncached.csv"
        os.makedirs(os.path.dirname(csv_path), exist_ok=True)
        with open(csv_path, "w", newline="") as f:
            writer = csv.DictWriter(
                f,
                fieldnames=["key_size","cores","avg_enc","std_enc","avg_add","std_add","accuracy"]
            )
            writer.writeheader()
            writer.writerows(results)
        print(f"All results written to {csv_path}")

if __name__ == "__main__":
    main()

Overwriting mpi_paillier_homomorphic_experiment_uncached.py


In [ ]:
!mpirun --allow-run-as-root --oversubscribe -np 32 stdbuf -oL python mpi_paillier_homomorphic_experiment_uncached.py

== Paillier Experiment Uncached ==

-- Generating 512-bit key pair --
Experiment with 4 cores:
  Encryption: avg = 20.339023s, std = 0.051418s
  Addition:   avg = 0.045683s, std = 0.000445s
  Accuracy:   100.0% (expected sum = 536887296)

Experiment with 8 cores:
  Encryption: avg = 10.217160s, std = 0.031148s
  Addition:   avg = 0.028102s, std = 0.010064s
  Accuracy:   100.0% (expected sum = 536887296)

Experiment with 16 cores:
  Encryption: avg = 5.086898s, std = 0.015255s
  Addition:   avg = 0.028069s, std = 0.027684s
  Accuracy:   100.0% (expected sum = 536887296)

Experiment with 32 cores:
  Encryption: avg = 2.545882s, std = 0.020799s
  Addition:   avg = 0.013954s, std = 0.003554s
  Accuracy:   100.0% (expected sum = 536887296)


-- Generating 1024-bit key pair --
Experiment with 4 cores:
  Encryption: avg = 127.601916s, std = 0.195703s
  Addition:   avg = 0.205375s, std = 0.107994s
  Accuracy:   100.0% (expected sum = 536887296)

Experiment with 8 cores:
  Encryption: avg = 63.

#2. Paillier Cached

In [ ]:
%%writefile mpi_paillier_homomorphic_experiment_cached.py

from mpi4py import MPI
import numpy as np
from phe import paillier
import csv
import os
import math

def encryption_experiment(comm, total_numbers, radices, zero_cipher):
    """
    Rache-style encryption: build each ciphertext by summing
    cached encryptions of powers of two.
    """
    rank = comm.Get_rank()
    size = comm.Get_size()
    # scatter the data as before
    data = np.arange(1, total_numbers + 1, dtype='int') if rank == 0 else None
    chunk_size = total_numbers // size
    chunk = np.empty(chunk_size, dtype='int')
    comm.Scatter(data, chunk, root=0)

    # encrypt by summing cached radices based on x's bits
    start_time = MPI.Wtime()
    local_encrypted_chunk = []
    for x in chunk:
        cipher = zero_cipher
        # for each bit position
        for j, rct in enumerate(radices):
            if (x >> j) & 1:
                cipher = cipher + rct
        local_encrypted_chunk.append(cipher)
    encryption_time = MPI.Wtime() - start_time

    return encryption_time, local_encrypted_chunk

def homomorphic_addition_experiment(comm, local_encrypted_chunk, private_key):
    # unchanged from before
    start_time = MPI.Wtime()
    local_sum = local_encrypted_chunk[0]
    for ct in local_encrypted_chunk[1:]:
        local_sum = local_sum + ct

    gathered = comm.gather(local_sum, root=0)
    if comm.Get_rank() == 0:
        final = gathered[0]
        for s in gathered[1:]:
            final = final + s
        decrypted = private_key.decrypt(final)
    else:
        decrypted = None

    addition_time = MPI.Wtime() - start_time
    return addition_time, decrypted

def build_cache(public_key, bit_len):
    """
    Measure the time to build the Rache cache of encryptions of powers of two.
    """
    start_time = MPI.Wtime()
    # encrypt powers of two 2**0, 2**1, ..., 2**(bit_len-1)
    radices = [public_key.encrypt(2**i) for i in range(bit_len)]
    zero_cipher = public_key.encrypt(0)
    cache_time = MPI.Wtime() - start_time

    return cache_time, radices, zero_cipher

def run_experiment(exp_cores, total_numbers, public_key, private_key, world_comm, iterations=5):
    rank = world_comm.Get_rank()
    size = world_comm.Get_size()

    # split communicator for this core count
    if size == exp_cores:
        sub_comm = world_comm
    else:
        color = 0 if rank < exp_cores else MPI.UNDEFINED
        sub_comm = world_comm.Split(color, rank)
    if sub_comm == MPI.COMM_NULL:
        return None

    # --- Rache initialization (once per sub-comm) ---
    # use radix = 2
    m = total_numbers
    bit_len = m.bit_length()  # number of bits needed

    # Time the cache building step
    if sub_comm.Get_rank() == 0:
        cache_time, radices, zero_cipher = build_cache(public_key, bit_len)
    else:
        cache_time, radices, zero_cipher = None, None, None

    # Broadcast the cache to all processes in the sub-communicator
    cache_time = sub_comm.bcast(cache_time, root=0)
    radices = sub_comm.bcast(radices, root=0)
    zero_cipher = sub_comm.bcast(zero_cipher, root=0)

    enc_times, add_times, decrypted_list = [], [], []

    for _ in range(iterations):
        et, enc_chunk = encryption_experiment(sub_comm, total_numbers, radices, zero_cipher)
        at, dec = homomorphic_addition_experiment(sub_comm, enc_chunk, private_key)
        enc_times.append(et)
        add_times.append(at)
        if sub_comm.Get_rank() == 0:
            decrypted_list.append(dec)

    if sub_comm.Get_rank() == 0:
        avg_enc = np.mean(enc_times)
        std_enc = np.std(enc_times)
        avg_add = np.mean(add_times)
        std_add = np.std(add_times)
        expected_sum = total_numbers * (total_numbers + 1) // 2
        accuracy = sum(1 for dec in decrypted_list if dec == expected_sum) / len(decrypted_list)

        print(f"Experiment with {exp_cores} cores:")
        print(f"  Cache Building: {cache_time:.6f}s")
        print(f"  Encryption: avg = {avg_enc:.6f}s, std = {std_enc:.6f}s")
        print(f"  Addition:   avg = {avg_add:.6f}s, std = {std_add:.6f}s")
        print(f"  Accuracy:   {accuracy*100:.1f}% (expected sum = {expected_sum})\n")

        return {
            "cores": exp_cores,
            "cache_time": cache_time,
            "avg_enc": avg_enc,
            "std_enc": std_enc,
            "avg_add": avg_add,
            "std_add": std_add,
            "accuracy": accuracy
        }
    return None

def main():
    world = MPI.COMM_WORLD
    rank = world.Get_rank()
    size = world.Get_size()

    total_numbers = 32768
    experiment_core_counts = [4, 8, 16, 32]
    key_sizes = [512, 1024]
    iterations = 10
    results = []

    if rank == 0:
        print("== Paillier Experiment Cached ==")

    for n_bits in key_sizes:
        if rank == 0:
            print(f"\n-- Generating {n_bits}-bit key pair --")
            public_key, private_key = paillier.generate_paillier_keypair(n_length=n_bits)
        else:
            public_key = private_key = None

        public_key = world.bcast(public_key, root=0)
        private_key = world.bcast(private_key, root=0)

        for c in experiment_core_counts:
            if size >= c:
                res = run_experiment(c, total_numbers, public_key, private_key, world, iterations)
                if rank == 0 and res:
                    res["key_size"] = n_bits
                    results.append(res)
            elif rank == 0:
                print(f"Skipping {c} cores (only {size} available)")

    if rank == 0 and results:
        csv_path = "/content/drive/MyDrive/praxisfiles/Benchmark/paillier_experiment_cached.csv"
        os.makedirs(os.path.dirname(csv_path), exist_ok=True)
        with open(csv_path, "w", newline="") as f:
            writer = csv.DictWriter(
                f,
                fieldnames=["key_size","cores","cache_time","avg_enc","std_enc","avg_add","std_add","accuracy"]
            )
            writer.writeheader()
            writer.writerows(results)
        print(f"All results written to {csv_path}")

if __name__ == "__main__":
    main()

Overwriting mpi_paillier_homomorphic_experiment_cached.py


In [ ]:
!mpirun --allow-run-as-root --oversubscribe -np 32 stdbuf -oL python mpi_paillier_homomorphic_experiment_cached.py

== Paillier Experiment Cached ==

-- Generating 512-bit key pair --
Experiment with 4 cores:
  Cache Building: 0.040552s
  Encryption: avg = 0.313139s, std = 0.002051s
  Addition:   avg = 0.134364s, std = 0.002467s
  Accuracy:   100.0% (expected sum = 536887296)

Experiment with 8 cores:
  Cache Building: 0.040379s
  Encryption: avg = 0.144719s, std = 0.001469s
  Addition:   avg = 0.091272s, std = 0.002064s
  Accuracy:   100.0% (expected sum = 536887296)

Experiment with 16 cores:
  Cache Building: 0.040595s
  Encryption: avg = 0.067184s, std = 0.000821s
  Addition:   avg = 0.056201s, std = 0.001294s
  Accuracy:   100.0% (expected sum = 536887296)

Experiment with 32 cores:
  Cache Building: 0.045299s
  Encryption: avg = 0.031896s, std = 0.000830s
  Addition:   avg = 0.035128s, std = 0.000715s
  Accuracy:   100.0% (expected sum = 536887296)


-- Generating 1024-bit key pair --
Experiment with 4 cores:
  Cache Building: 0.284455s
  Encryption: avg = 0.855060s, std = 0.011145s
  Addition

#3. BFV Cached

In [ ]:
%%writefile mpi_tenseal_bfv_experiment_uncached.py

from mpi4py import MPI
import numpy as np
import tenseal as ts
import csv
import os

def encryption_experiment(comm, total_numbers, ctx):
    """
    Direct BFV encryption: encrypt each number directly without using a cache.
    """
    rank = comm.Get_rank()
    size = comm.Get_size()
    data = np.arange(1, total_numbers + 1, dtype='int') if rank == 0 else None
    chunk_size = total_numbers // size
    chunk = np.empty(chunk_size, dtype='int')
    comm.Scatter(data, chunk, root=0)

    start = MPI.Wtime()
    local_ct = []
    for x in chunk:
        cipher = ts.bfv_vector(ctx, [x])
        local_ct.append(cipher)
    enc_time = MPI.Wtime() - start

    return enc_time, local_ct

def homomorphic_addition_experiment(comm, local_ct, ctx, plain_modulus):
    start = MPI.Wtime()
    # local sum
    local_sum = local_ct[0]
    for ct in local_ct[1:]:
        local_sum += ct
    # gather via serialization
    serialized = local_sum.serialize()
    collected = comm.gather(serialized, root=0)

    if comm.Get_rank() == 0:
        final = ts.bfv_vector_from(ctx, collected[0])
        for s in collected[1:]:
            final += ts.bfv_vector_from(ctx, s)
        dec = final.decrypt()[0]
        # adjust if negative
        dec_mod = dec if dec >= 0 else dec + plain_modulus
    else:
        dec_mod = None

    add_time = MPI.Wtime() - start
    return add_time, dec_mod

def run_experiment(exp_cores, total_numbers, ctx, plain_modulus, world_comm, iterations, key_size):
    rank = world_comm.Get_rank()
    size = world_comm.Get_size()

    # carve out exactly exp_cores processes
    if size == exp_cores:
        sub_comm = world_comm
    else:
        color = 0 if rank < exp_cores else MPI.UNDEFINED
        sub_comm = world_comm.Split(color, rank)
    if sub_comm == MPI.COMM_NULL:
        return None

    enc_times, add_times, decs = [], [], []

    for _ in range(iterations):
        et, ct = encryption_experiment(sub_comm, total_numbers, ctx)
        at, dec = homomorphic_addition_experiment(sub_comm, ct, ctx, plain_modulus)
        enc_times.append(et)
        add_times.append(at)
        if sub_comm.Get_rank() == 0:
            decs.append(dec)

    if sub_comm.Get_rank() == 0:
        avg_enc = np.mean(enc_times)
        std_enc = np.std(enc_times)
        avg_add = np.mean(add_times)
        std_add = np.std(add_times)
        expected = total_numbers * (total_numbers + 1) // 2
        accuracy = sum(1 for d in decs if d == expected) / len(decs)
        print(
            f"key_size={key_size} cores={exp_cores} → "
            f"enc={avg_enc:.6f}s±{std_enc:.6f}, "
            f"add={avg_add:.6f}s±{std_add:.6f}, "
            f"acc={accuracy*100:.1f}%"
        )
        return {
            "key_size": key_size,
            "cores": exp_cores,
            "avg_enc": avg_enc,
            "std_enc": std_enc,
            "avg_add": avg_add,
            "std_add": std_add,
            "accuracy": accuracy
        }
    return None

def main():
    world = MPI.COMM_WORLD
    rank = world.Get_rank()
    size = world.Get_size()

    total_numbers = 32768
    core_counts = [4, 8, 16, 32]
    iterations = 10

    # BFV parameter grid - map to key sizes 512 and 1024 for output
    param_grid = [
        (4096, [30, 30, 30], 512),
        (8192, [40, 40, 40], 1024),
    ]

    if rank == 0:
        print("== BFV tenSEAL uncached ==")
        all_results = []

    for poly_deg, coeff_bits, key_size in param_grid:
        if rank == 0:
            print(f"\n-- params: key_size={key_size}, poly_modulus_degree={poly_deg}, coeff_mod_bit_sizes={coeff_bits} --")
            ctx = ts.context(
                ts.SCHEME_TYPE.BFV,
                poly_modulus_degree=poly_deg,
                plain_modulus=536903681,
                coeff_mod_bit_sizes=coeff_bits
            )
            serialized = ctx.serialize(save_secret_key=True)
        else:
            serialized = None

        serialized = world.bcast(serialized, root=0)
        ctx = ts.context_from(serialized)
        plain_modulus = world.bcast(536903681, root=0)

        for c in core_counts:
            if size >= c:
                res = run_experiment(c, total_numbers, ctx, plain_modulus, world, iterations, key_size)
                if rank == 0 and res:
                    all_results.append(res)
            elif rank == 0:
                print(f"skip cores={c} (only {size} avail)")

    if rank == 0 and all_results:
        out_path = "/content/drive/MyDrive/praxisfiles/Benchmark/bfv_experiment_uncached.csv"
        os.makedirs(os.path.dirname(out_path), exist_ok=True)
        with open(out_path, "w", newline="") as f:
            cols = [
                "key_size", "cores",
                "avg_enc", "std_enc", "avg_add", "std_add", "accuracy"
            ]
            writer = csv.DictWriter(f, fieldnames=cols)
            writer.writeheader()
            writer.writerows(all_results)
        print(f"\nResults saved to {out_path}")

if __name__ == "__main__":
    main()

Overwriting mpi_tenseal_bfv_experiment_uncached.py


In [ ]:
!mpirun --allow-run-as-root --oversubscribe -np 32 stdbuf -oL python mpi_tenseal_bfv_experiment_uncached.py

== BFV tenSEAL uncached ==

-- params: key_size=512, poly_modulus_degree=4096, coeff_mod_bit_sizes=[30, 30, 30] --
key_size=512 cores=4 → enc=16.225673s±0.257165, add=0.415460s±0.098837, acc=100.0%
key_size=512 cores=8 → enc=8.049549s±0.031846, add=0.265352s±0.132753, acc=100.0%
key_size=512 cores=16 → enc=4.031643s±0.041724, add=0.130315s±0.043490, acc=100.0%
key_size=512 cores=32 → enc=2.017529s±0.012871, add=0.154922s±0.059795, acc=100.0%

-- params: key_size=1024, poly_modulus_degree=8192, coeff_mod_bit_sizes=[40, 40, 40] --
key_size=1024 cores=4 → enc=32.500552s±0.336115, add=0.708878s±0.055695, acc=100.0%
key_size=1024 cores=8 → enc=16.187879s±0.023471, add=0.524597s±0.155321, acc=100.0%
key_size=1024 cores=16 → enc=8.093628s±0.019111, add=0.484714s±0.094637, acc=100.0%
key_size=1024 cores=32 → enc=4.088705s±0.020135, add=0.279439s±0.041103, acc=100.0%

Results saved to /content/drive/MyDrive/praxisfiles/Benchmark/bfv_experiment_uncached.csv


# 4. BFV Cached


In [ ]:
%%writefile mpi_tenseal_bfv_experiment_cached.py

from mpi4py import MPI
import numpy as np
import tenseal as ts
import csv
import os

def build_radix_cache_experiment(ctx, total_numbers):
    """
    Measure the time to build the radix cache.
    """
    start = MPI.Wtime()
    # use radix = 2
    bit_len = total_numbers.bit_length()
    radices = [ts.bfv_vector(ctx, [2**i]) for i in range(bit_len)]
    zero_cipher = ts.bfv_vector(ctx, [0])
    cache_time = MPI.Wtime() - start

    return cache_time, radices, zero_cipher

def encryption_experiment(comm, total_numbers, radices, zero_cipher):
    """
    Rache-style BFV encryption: for each x, sum the cached encryptions
    of powers of two corresponding to x's binary bits.
    """
    rank = comm.Get_rank()
    size = comm.Get_size()
    data = np.arange(1, total_numbers + 1, dtype='int') if rank == 0 else None
    chunk_size = total_numbers // size
    chunk = np.empty(chunk_size, dtype='int')
    comm.Scatter(data, chunk, root=0)

    start = MPI.Wtime()
    local_ct = []
    for x in chunk:
        # start from zero
        cipher = zero_cipher
        # add each power-of-two if bit is set
        for j, rct in enumerate(radices):
            if (x >> j) & 1:
                cipher = cipher + rct
        local_ct.append(cipher)
    enc_time = MPI.Wtime() - start

    return enc_time, local_ct

def homomorphic_addition_experiment(comm, local_ct, ctx, plain_modulus):
    start = MPI.Wtime()
    # local sum
    local_sum = local_ct[0]
    for ct in local_ct[1:]:
        local_sum += ct
    # gather via serialization
    serialized = local_sum.serialize()
    collected = comm.gather(serialized, root=0)

    if comm.Get_rank() == 0:
        final = ts.bfv_vector_from(ctx, collected[0])
        for s in collected[1:]:
            final += ts.bfv_vector_from(ctx, s)
        dec = final.decrypt()[0]
        # adjust if negative
        dec_mod = dec if dec >= 0 else dec + plain_modulus
    else:
        dec_mod = None

    add_time = MPI.Wtime() - start
    return add_time, dec_mod

def run_experiment(exp_cores, total_numbers, ctx, plain_modulus, world_comm, iterations, key_size):
    rank = world_comm.Get_rank()
    size = world_comm.Get_size()

    # carve out exactly exp_cores processes
    if size == exp_cores:
        sub_comm = world_comm
    else:
        color = 0 if rank < exp_cores else MPI.UNDEFINED
        sub_comm = world_comm.Split(color, rank)
    if sub_comm == MPI.COMM_NULL:
        return None

    # --- Measure radix cache building time ---
    cache_time, radices, zero_cipher = build_radix_cache_experiment(ctx, total_numbers)
    # Synchronize the cache time across processes
    cache_times = sub_comm.gather(cache_time, root=0)
    if sub_comm.Get_rank() == 0:
        avg_cache_time = np.mean(cache_times)
    else:
        avg_cache_time = None

    enc_times, add_times, decs = [], [], []

    for _ in range(iterations):
        et, ct = encryption_experiment(sub_comm, total_numbers, radices, zero_cipher)
        at, dec = homomorphic_addition_experiment(sub_comm, ct, ctx, plain_modulus)
        enc_times.append(et)
        add_times.append(at)
        if sub_comm.Get_rank() == 0:
            decs.append(dec)

    if sub_comm.Get_rank() == 0:
        avg_enc = np.mean(enc_times)
        std_enc = np.std(enc_times)
        avg_add = np.mean(add_times)
        std_add = np.std(add_times)
        expected = total_numbers * (total_numbers + 1) // 2
        accuracy = sum(1 for d in decs if d == expected) / len(decs)
        print(
            f"key_size={key_size} cores={exp_cores} → "  # Fixed: Now using key_size parameter
            f"cache_time={avg_cache_time:.6f}s, "
            f"enc={avg_enc:.6f}s±{std_enc:.6f}, "
            f"add={avg_add:.6f}s±{std_add:.6f}, "
            f"acc={accuracy*100:.1f}%"
        )
        return {
            "key_size": key_size,
            "cores": exp_cores,
            "cache_time": avg_cache_time,
            "avg_enc": avg_enc,
            "std_enc": std_enc,
            "avg_add": avg_add,
            "std_add": std_add,
            "accuracy": accuracy
        }
    return None

def main():
    world = MPI.COMM_WORLD
    rank = world.Get_rank()
    size = world.Get_size()

    total_numbers = 32768
    core_counts = [4, 8, 16, 32]
    iterations = 10

    # BFV parameter grid - map to key sizes 512 and 1024 for output
    param_grid = [
        (4096, [30, 30, 30], 512),
        (8192, [40, 40, 40], 1024),
    ]

    if rank == 0:
        print("== BFV tenSEAL cached ==")
        all_results = []

    for poly_deg, coeff_bits, key_size in param_grid:
        if rank == 0:
            print(f"\n-- params: key_size={key_size}, poly_modulus_degree={poly_deg}, coeff_mod_bit_sizes={coeff_bits} --")
            ctx = ts.context(
                ts.SCHEME_TYPE.BFV,
                poly_modulus_degree=poly_deg,
                plain_modulus=536903681,
                coeff_mod_bit_sizes=coeff_bits
            )
            serialized = ctx.serialize(save_secret_key=True)
        else:
            serialized = None

        serialized = world.bcast(serialized, root=0)
        ctx = ts.context_from(serialized)
        plain_modulus = world.bcast(536903681, root=0)

        for c in core_counts:
            if size >= c:
                # Pass key_size to run_experiment
                res = run_experiment(c, total_numbers, ctx, plain_modulus, world, iterations, key_size)
                if rank == 0 and res:
                    all_results.append(res)
            elif rank == 0:
                print(f"skip cores={c} (only {size} avail)")

    if rank == 0 and all_results:
        out_path = "/content/drive/MyDrive/praxisfiles/Benchmark/bfv_experiment_cached.csv"
        os.makedirs(os.path.dirname(out_path), exist_ok=True)
        with open(out_path, "w", newline="") as f:
            cols = [
                "key_size", "cores", "cache_time",
                "avg_enc", "std_enc", "avg_add", "std_add", "accuracy"
            ]
            writer = csv.DictWriter(f, fieldnames=cols)
            writer.writeheader()
            writer.writerows(all_results)
        print(f"\nResults saved to {out_path}")

if __name__ == "__main__":
    main()

Overwriting mpi_tenseal_bfv_experiment_cached.py


In [ ]:
!mpirun --allow-run-as-root --oversubscribe -np 32 stdbuf -oL python mpi_tenseal_bfv_experiment_cached.py

== BFV tenSEAL cached ==

-- params: key_size=512, poly_modulus_degree=4096, coeff_mod_bit_sizes=[30, 30, 30] --
key_size=512 cores=4 → cache_time=0.044142s, enc=2.490427s±0.180014, add=1.032264s±0.034374, acc=100.0%
key_size=512 cores=8 → cache_time=0.035390s, enc=1.133357s±0.029190, add=0.746909s±0.135542, acc=100.0%
key_size=512 cores=16 → cache_time=0.035743s, enc=0.518082s±0.006710, add=0.462601s±0.047257, acc=100.0%
key_size=512 cores=32 → cache_time=0.035001s, enc=0.250262s±0.010746, add=0.301199s±0.022499, acc=100.0%

-- params: key_size=1024, poly_modulus_degree=8192, coeff_mod_bit_sizes=[40, 40, 40] --
key_size=1024 cores=4 → cache_time=0.081236s, enc=4.913921s±0.367111, add=1.995159s±0.054632, acc=100.0%
key_size=1024 cores=8 → cache_time=0.069026s, enc=2.208861s±0.014033, add=1.468577s±0.196802, acc=100.0%
key_size=1024 cores=16 → cache_time=0.073567s, enc=1.039178s±0.009230, add=0.917910s±0.087568, acc=100.0%
key_size=1024 cores=32 → cache_time=0.070282s, enc=0.518909s±0.0

# 5. CKKS Uncached

In [ ]:
%%writefile mpi_tenseal_ckks_experiment_uncached.py

from mpi4py import MPI
import numpy as np
import tenseal as ts
import csv
import os

def encryption_experiment(comm, total_numbers, ctx):
    """
    Direct CKKS encryption: encrypt each integer directly without caching.
    """
    rank = comm.Get_rank()
    size = comm.Get_size()
    # scatter the integers
    data = np.arange(1, total_numbers + 1, dtype='int') if rank == 0 else None
    chunk_size = total_numbers // size
    chunk = np.empty(chunk_size, dtype='int')
    comm.Scatter(data, chunk, root=0)

    start = MPI.Wtime()
    local_ct = []
    for x in chunk:
        # No cache, directly encrypt each number
        cipher = ts.ckks_vector(ctx, [float(x)])
        local_ct.append(cipher)
    enc_time = MPI.Wtime() - start

    return enc_time, local_ct

def homomorphic_addition_experiment(comm, local_ct, ctx):
    """
    Sum locally, gather, rehydrate and sum on rank 0, then decrypt.
    """
    start = MPI.Wtime()
    local_sum = local_ct[0]
    for ct in local_ct[1:]:
        local_sum += ct

    serialized = local_sum.serialize()
    gathered = comm.gather(serialized, root=0)

    if comm.Get_rank() == 0:
        final = ts.ckks_vector_from(ctx, gathered[0])
        for s in gathered[1:]:
            final += ts.ckks_vector_from(ctx, s)
        dec = final.decrypt()[0]
    else:
        dec = None

    add_time = MPI.Wtime() - start
    return add_time, dec

def run_experiment(exp_cores, total_numbers, ctx, world_comm, iterations):
    rank = world_comm.Get_rank()
    size = world_comm.Get_size()

    # carve out exactly exp_cores processes
    if size == exp_cores:
        sub = world_comm
    else:
        color = 0 if rank < exp_cores else MPI.UNDEFINED
        sub = world_comm.Split(color, rank)
    if sub == MPI.COMM_NULL:
        return None

    enc_times, add_times, decs = [], [], []

    for _ in range(iterations):
        et, cts = encryption_experiment(sub, total_numbers, ctx)
        at, dec = homomorphic_addition_experiment(sub, cts, ctx)
        enc_times.append(et)
        add_times.append(at)
        if sub.Get_rank() == 0:
            decs.append(dec)

    if sub.Get_rank() == 0:
        avg_enc = np.mean(enc_times)
        std_enc = np.std(enc_times)
        avg_add = np.mean(add_times)
        std_add = np.std(add_times)
        expected = total_numbers * (total_numbers + 1) // 2
        accuracy = 1.0  # Default perfect accuracy
        if abs(expected) > 0:
            accuracy = 1.0 - min(1.0, abs(decs[0] - expected) / expected)

        # More readable output format
        acc_percentage = accuracy * 100.0
        print(
            f"key_size={params['poly_modulus_degree']} cores={exp_cores} → "
            f"enc={avg_enc:.6f}s±{std_enc:.6f}, "
            f"add={avg_add:.6f}s±{std_add:.6f}, "
            f"acc={acc_percentage:.1f}%"
        )

        return {
            "key_size": params['poly_modulus_degree'],
            "cores": exp_cores,
            "avg_enc": avg_enc,
            "std_enc": std_enc,
            "avg_add": avg_add,
            "std_add": std_add,
            "accuracy": accuracy,
            "decrypted": decs[0],
            "expected": expected
        }
    return None

def main():
    world = MPI.COMM_WORLD
    rank = world.Get_rank()
    size = world.Get_size()

    total_numbers = 32768
    core_counts = [4, 8, 16, 32]
    iterations = 10

    # CKKS parameter grid
    param_grid = [
        {
            "poly_modulus_degree": 4096,
            "coeff_mod_bit_sizes": [40, 40],
            "global_scale": 2**20
        },
        {
            "poly_modulus_degree": 8192,
            "coeff_mod_bit_sizes": [60, 40, 40, 60],
            "global_scale": 2**25
        }
    ]

    global params  # Make params accessible in run_experiment

    if rank == 0:
        print("== CKKS tenSEAL uncached ==")
        all_results = []

    for params in param_grid:
        if rank == 0:
            print(
                f"\n-- params: key_size={params['poly_modulus_degree']}, "
                f"poly_modulus_degree={params['poly_modulus_degree']}, "
                f"coeff_mod_bit_sizes={params['coeff_mod_bit_sizes']} --"
            )
            ctx = ts.context(
                ts.SCHEME_TYPE.CKKS,
                poly_modulus_degree=params["poly_modulus_degree"],
                coeff_mod_bit_sizes=params["coeff_mod_bit_sizes"]
            )
            ctx.global_scale = params["global_scale"]
            serialized = ctx.serialize(save_secret_key=True)
        else:
            serialized = None

        serialized = world.bcast(serialized, root=0)
        ctx = ts.context_from(serialized)

        for c in core_counts:
            if size >= c:
                res = run_experiment(c, total_numbers, ctx, world, iterations)
                if rank == 0 and res:
                    all_results.append(res)
            elif rank == 0:
                # Skip silently as per requested output format
                pass

    if rank == 0 and all_results:
        out = "/content/drive/MyDrive/praxisfiles/Benchmark/ckks_experiment_uncached.csv"
        os.makedirs(os.path.dirname(out), exist_ok=True)

        # Fix: Include all fields from the results dictionary
        fields = ["key_size", "cores", "avg_enc", "std_enc", "avg_add", "std_add",
                  "accuracy", "decrypted", "expected"]

        # Write CSV with proper column headers
        with open(out, "w", newline="") as f:
            writer = csv.DictWriter(f, fieldnames=fields)
            writer.writeheader()
            writer.writerows(all_results)

        print(f"\nResults saved to {out}")

if __name__ == "__main__":
    main()

Overwriting mpi_tenseal_ckks_experiment_uncached.py


In [ ]:
!mpirun --allow-run-as-root --oversubscribe -np 32 stdbuf -oL python mpi_tenseal_ckks_experiment_uncached.py

== CKKS tenSEAL uncached ==

-- params: key_size=4096, poly_modulus_degree=4096, coeff_mod_bit_sizes=[40, 40] --
key_size=4096 cores=4 → enc=15.799881s±0.136776, add=0.188509s±0.037339, acc=0.0%
key_size=4096 cores=8 → enc=7.869833s±0.024360, add=0.150295s±0.074809, acc=0.0%
key_size=4096 cores=16 → enc=3.944253s±0.029594, add=0.104370s±0.033136, acc=0.0%
key_size=4096 cores=32 → enc=1.965047s±0.011517, add=0.108726s±0.014240, acc=0.0%

-- params: key_size=8192, poly_modulus_degree=8192, coeff_mod_bit_sizes=[60, 40, 40, 60] --
key_size=8192 cores=4 → enc=52.316360s±0.509640, add=0.770488s±0.092504, acc=100.0%
key_size=8192 cores=8 → enc=26.087777s±0.054856, add=0.584487s±0.255770, acc=100.0%
key_size=8192 cores=16 → enc=13.099003s±0.024367, add=0.302661s±0.139970, acc=100.0%
key_size=8192 cores=32 → enc=6.641759s±0.027219, add=0.209245s±0.064417, acc=100.0%

Results saved to /content/drive/MyDrive/praxisfiles/Benchmark/ckks_experiment_uncached.csv


# 6. CKKS Cached

In [ ]:
%%writefile mpi_tenseal_ckks_experiment_cached.py

from mpi4py import MPI
import numpy as np
import tenseal as ts
import csv
import os

def build_radix_cache(ctx, bit_len):
    """
    Build the cache of radix encryptions and measure the time taken.
    """
    start = MPI.Wtime()
    radices = [ts.ckks_vector(ctx, [float(2**i)]) for i in range(bit_len)]
    zero_cipher = ts.ckks_vector(ctx, [0.0])
    cache_time = MPI.Wtime() - start
    return cache_time, radices, zero_cipher

def encryption_experiment(comm, total_numbers, radices, zero_cipher):
    """
    Rache-style CKKS encryption: for each integer x, build its encryption
    by summing cached encryptions of powers of two corresponding to x's bits.
    """
    rank = comm.Get_rank()
    size = comm.Get_size()
    # scatter the integers
    data = np.arange(1, total_numbers + 1, dtype='int') if rank == 0 else None
    chunk_size = total_numbers // size
    chunk = np.empty(chunk_size, dtype='int')
    comm.Scatter(data, chunk, root=0)

    start = MPI.Wtime()
    local_ct = []
    for x in chunk:
        cipher = zero_cipher
        # add each power-of-two cipher if that bit is set
        for j, rct in enumerate(radices):
            if (x >> j) & 1:
                cipher = cipher + rct
        local_ct.append(cipher)
    enc_time = MPI.Wtime() - start

    return enc_time, local_ct

def homomorphic_addition_experiment(comm, local_ct, ctx):
    """
    Sum locally, gather, rehydrate and sum on rank 0, then decrypt.
    """
    start = MPI.Wtime()
    local_sum = local_ct[0]
    for ct in local_ct[1:]:
        local_sum += ct

    serialized = local_sum.serialize()
    gathered = comm.gather(serialized, root=0)

    if comm.Get_rank() == 0:
        final = ts.ckks_vector_from(ctx, gathered[0])
        for s in gathered[1:]:
            final += ts.ckks_vector_from(ctx, s)
        dec = final.decrypt()[0]
    else:
        dec = None

    add_time = MPI.Wtime() - start
    return add_time, dec

def run_experiment(exp_cores, total_numbers, ctx, world_comm, iterations):
    rank = world_comm.Get_rank()
    size = world_comm.Get_size()

    # carve out exactly exp_cores processes
    if size == exp_cores:
        sub = world_comm
    else:
        color = 0 if rank < exp_cores else MPI.UNDEFINED
        sub = world_comm.Split(color, rank)
    if sub == MPI.COMM_NULL:
        return None

    # Measure cache building time on rank 0 and broadcast
    bit_len = total_numbers.bit_length()
    if sub.Get_rank() == 0:
        cache_time, radices, zero_cipher = build_radix_cache(ctx, bit_len)
    else:
        cache_time, radices, zero_cipher = None, None, None

    # Broadcast cache time and cached values
    cache_time = sub.bcast(cache_time, root=0)

    # Need to build the cache on all ranks even though we only time it on rank 0
    if sub.Get_rank() != 0:
        _, radices, zero_cipher = build_radix_cache(ctx, bit_len)

    enc_times, add_times, decs = [], [], []

    for _ in range(iterations):
        et, cts = encryption_experiment(sub, total_numbers, radices, zero_cipher)
        at, dec = homomorphic_addition_experiment(sub, cts, ctx)
        enc_times.append(et)
        add_times.append(at)
        if sub.Get_rank() == 0:
            decs.append(dec)

    if sub.Get_rank() == 0:
        avg_enc = np.mean(enc_times)
        std_enc = np.std(enc_times)
        avg_add = np.mean(add_times)
        std_add = np.std(add_times)
        expected = total_numbers * (total_numbers + 1) // 2
        accuracy = 1.0  # Default perfect accuracy
        if abs(expected) > 0:
            accuracy = 1.0 - min(1.0, abs(decs[0] - expected) / expected)

        # More readable output format
        acc_percentage = accuracy * 100.0
        print(
            f"key_size={params['poly_modulus_degree']} cores={exp_cores} → "
            f"cache_time={cache_time:.6f}s, "
            f"enc={avg_enc:.6f}s±{std_enc:.6f}, "
            f"add={avg_add:.6f}s±{std_add:.6f}, "
            f"acc={acc_percentage:.1f}%"
        )

        return {
            "key_size": params['poly_modulus_degree'],
            "cores": exp_cores,
            "cache_time": cache_time,
            "avg_enc": avg_enc,
            "std_enc": std_enc,
            "avg_add": avg_add,
            "std_add": std_add,
            "accuracy": accuracy,
            "decrypted": decs[0],
            "expected": expected
        }
    return None

def main():
    world = MPI.COMM_WORLD
    rank = world.Get_rank()
    size = world.Get_size()

    total_numbers = 32768
    core_counts = [4, 8, 16, 32]
    iterations = 10

    # CKKS parameter grid
    param_grid = [
        {
            "poly_modulus_degree": 4096,
            "coeff_mod_bit_sizes": [40, 40],
            "global_scale": 2**20
        },
        {
            "poly_modulus_degree": 8192,
            "coeff_mod_bit_sizes": [60, 40, 40, 60],
            "global_scale": 2**25
        }
    ]

    global params  # Make params accessible in run_experiment

    if rank == 0:
        print("== CKKS tenSEAL cached ==")
        all_results = []

    for params in param_grid:
        if rank == 0:
            print(
                f"\n-- params: key_size={params['poly_modulus_degree']}, "
                f"poly_modulus_degree={params['poly_modulus_degree']}, "
                f"coeff_mod_bit_sizes={params['coeff_mod_bit_sizes']} --"
            )
            ctx = ts.context(
                ts.SCHEME_TYPE.CKKS,
                poly_modulus_degree=params["poly_modulus_degree"],
                coeff_mod_bit_sizes=params["coeff_mod_bit_sizes"]
            )
            ctx.global_scale = params["global_scale"]
            serialized = ctx.serialize(save_secret_key=True)
        else:
            serialized = None

        serialized = world.bcast(serialized, root=0)
        ctx = ts.context_from(serialized)

        for c in core_counts:
            if size >= c:
                res = run_experiment(c, total_numbers, ctx, world, iterations)
                if rank == 0 and res:
                    all_results.append(res)
            elif rank == 0:
                # Skip silently as per requested output format
                pass

    if rank == 0 and all_results:
        out = "/content/drive/MyDrive/praxisfiles/Benchmark/ckks_experiment_cached.csv"
        os.makedirs(os.path.dirname(out), exist_ok=True)
        with open(out, "w", newline="") as f:
            fields = [
                "key_size", "cores", "cache_time",
                "avg_enc", "std_enc", "avg_add", "std_add",
                "accuracy", "decrypted", "expected"
            ]
            writer = csv.DictWriter(f, fieldnames=fields)
            writer.writeheader()
            writer.writerows(all_results)
        print(f"\nResults saved to {out}")

if __name__ == "__main__":
    main()

Overwriting mpi_tenseal_ckks_experiment_cached.py


In [ ]:
!mpirun --allow-run-as-root --oversubscribe -np 32 stdbuf -oL python mpi_tenseal_ckks_experiment_cached.py

== CKKS tenSEAL cached ==

-- params: key_size=4096, poly_modulus_degree=4096, coeff_mod_bit_sizes=[40, 40] --
key_size=4096 cores=4 → cache_time=0.037645s, enc=1.074209s±0.086468, add=0.387846s±0.028895, acc=0.0%
key_size=4096 cores=8 → cache_time=0.036434s, enc=0.473331s±0.011142, add=0.293748s±0.041562, acc=0.0%
key_size=4096 cores=16 → cache_time=0.036924s, enc=0.227831s±0.009589, add=0.186749s±0.032520, acc=0.0%
key_size=4096 cores=32 → cache_time=0.033391s, enc=0.104738s±0.004854, add=0.115072s±0.013469, acc=0.0%

-- params: key_size=8192, poly_modulus_degree=8192, coeff_mod_bit_sizes=[60, 40, 40, 60] --
key_size=8192 cores=4 → cache_time=0.109066s, enc=5.881317s±0.530299, add=2.294595s±0.037594, acc=100.0%
key_size=8192 cores=8 → cache_time=0.119686s, enc=2.672085s±0.010021, add=1.665355s±0.259327, acc=100.0%
key_size=8192 cores=16 → cache_time=0.112493s, enc=1.296574s±0.010489, add=1.113295s±0.139755, acc=100.0%
key_size=8192 cores=32 → cache_time=0.110498s, enc=0.650193s±0.011

# Plots Uncached/Cached

In [ ]:
import pandas as pd
import plotly.express as px

# --- load & prepare ---
path = "/content/drive/MyDrive/praxisfiles/Benchmark"

bfv = pd.read_csv(f"{path}/bfv_experiment_uncached.csv").assign(scheme='BFV')
bfv.rename(columns={'poly_modulus_degree':'key_size'}, inplace=True)

paillier = pd.read_csv(f"{path}/paillier_experiment_uncached.csv").assign(scheme='Paillier')
# If needed: paillier.rename(columns={'poly_modulus_degree':'key_size'}, inplace=True)

ckks = pd.read_csv(f"{path}/ckks_experiment_uncached.csv").assign(scheme='CKKS')
ckks.rename(columns={'poly_modulus_degree':'key_size'}, inplace=True)

df = pd.concat([bfv, paillier, ckks], ignore_index=True)

# --- aggregate over key sizes for each scheme & core count ---
df_cpu = (
    df
    .groupby(['scheme','cores'])
    .agg(
        enc_time=('avg_enc','mean'),
        add_time=('avg_add','mean')
    )
    .reset_index()
)
df_cpu['cores_str'] = df_cpu['cores'].astype(str)

# --- Encryption time vs CPU cores ---
fig_enc = px.bar(
    df_cpu,
    x='cores_str',
    y='enc_time',
    color='scheme',
    barmode='group',
    labels={
        'cores_str':'CPU Cores',
        'enc_time':'Avg Encryption Time (s)',
        'scheme':'Scheme'
    },
    title='Uncached Encryption Time vs CPU Cores by Scheme'
)
fig_enc.update_layout(xaxis=dict(type='category'))
fig_enc.show()

# --- Addition time vs CPU cores ---
fig_add = px.bar(
    df_cpu,
    x='cores_str',
    y='add_time',
    color='scheme',
    barmode='group',
    labels={
        'cores_str':'CPU Cores',
        'add_time':'Avg Addition Time (s)',
        'scheme':'Scheme'
    },
    title='Uncached Addition Time vs CPU Cores by Scheme'
)
fig_add.update_layout(xaxis=dict(type='category'))
fig_add.show()


In [ ]:
import pandas as pd
import plotly.express as px

# Base path where your cached CSVs live
path = "/content/drive/MyDrive/praxisfiles/Benchmark"

# --- Load & tag cached data ---
def load_cached(fname, scheme_name):
    df = pd.read_csv(f"{path}/{fname}.csv")
    df['scheme'] = scheme_name
    if 'poly_modulus_degree' in df.columns:
        df.rename(columns={'poly_modulus_degree':'key_size'}, inplace=True)
    return df

bfv_cac     = load_cached("bfv_experiment_cached",     "BFV")
ckks_cac    = load_cached("ckks_experiment_cached",    "CKKS")
paillier_cac= load_cached("paillier_experiment_cached","Paillier")

# Combine
df = pd.concat([bfv_cac, ckks_cac, paillier_cac], ignore_index=True)

# Aggregate over key_size
df_cpu = (
    df
    .groupby(['scheme','cores'])
    .agg(
        enc_time=('avg_enc','mean'),
        add_time=('avg_add','mean')
    )
    .reset_index()
)
df_cpu['cores_str'] = df_cpu['cores'].astype(str)  # discrete X

# --- Encryption Time vs CPU Cores ---
fig_enc = px.bar(
    df_cpu,
    x='cores_str', y='enc_time',
    color='scheme',
    barmode='group',
    labels={
        'cores_str':'CPU Cores',
        'enc_time':'Encryption Time (s)',
        'scheme':'Scheme'
    },
    title='Cached Encryption Time vs CPU Cores'
)
fig_enc.update_xaxes(type='category')
fig_enc.show()

# --- Homomorphic Addition Time vs CPU Cores ---
fig_add = px.bar(
    df_cpu,
    x='cores_str', y='add_time',
    color='scheme',
    barmode='group',
    labels={
        'cores_str':'CPU Cores',
        'add_time':'Addition Time (s)',
        'scheme':'Scheme'
    },
    title='Cached Addition Time vs CPU Cores'
)
fig_add.update_xaxes(type='category')
fig_add.show()


In [ ]:
import pandas as pd
import plotly.graph_objects as go
import numpy as np

# Function to load and process data
def calculate_speedup_factors():
    algorithms = ["BFV", "CKKS", "PAILLIER"]
    core_values = [4, 8, 16, 32]
    results = []

    path = "/content/drive/MyDrive/praxisfiles/Benchmark"

    for algorithm in algorithms:
        try:
            # Load cached and uncached data
            cached_file = f"{path}/{algorithm.lower()}_experiment_cached.csv"
            uncached_file = f"{path}/{algorithm.lower()}_experiment_uncached.csv"

            cached_df = pd.read_csv(cached_file)
            uncached_df = pd.read_csv(uncached_file)

            # Process each core value
            for core in core_values:
                # Filter data for this core count
                cached_data = cached_df[cached_df["cores"] == core]
                uncached_data = uncached_df[uncached_df["cores"] == core]

                # Skip if no data for this core count
                if len(cached_data) == 0 or len(uncached_data) == 0:
                    continue

                # Calculate speedup for encryption
                cached_enc = cached_data["avg_enc"].mean()
                uncached_enc = uncached_data["avg_enc"].mean()
                speedup = uncached_enc / cached_enc

                # Store the result
                results.append({
                    "algorithm": algorithm,
                    "cores": core,
                    "speedup": speedup
                })

        except Exception as e:
            print(f"Error processing {algorithm}: {e}")

    return pd.DataFrame(results)

# Calculate speedup factors
speedup_df = calculate_speedup_factors()

# Create the plot
fig = go.Figure()

# Add bars for each algorithm
for algorithm in speedup_df["algorithm"].unique():
    algo_data = speedup_df[speedup_df["algorithm"] == algorithm]

    fig.add_trace(go.Bar(
        x=algo_data["cores"],
        y=algo_data["speedup"],
        name=algorithm,
        text=[f"{val:.1f}x" for val in algo_data["speedup"]],
        textposition="outside"
    ))

# Update layout
fig.update_layout(
    title="Speedup Factor (Uncached/Cached) by Algorithm and CPU Cores",
    xaxis=dict(
        title="CPU Cores",
        type="category",
        tickvals=[4, 8, 16, 32],
        ticktext=["4", "8", "16", "32"]
    ),
    yaxis=dict(
        title="Speedup Factor (Uncached/Cached)"
    ),
    barmode="group",
    legend=dict(
        title="Algorithm",
        orientation="h",
        yanchor="bottom",
        y=1.02,
        xanchor="right",
        x=1
    )
)

# Ensure y-axis starts at 0
fig.update_yaxes(range=[0, speedup_df["speedup"].max() * 1.2])

# Display the plot
fig.show()

In [ ]:
import pandas as pd
import plotly.graph_objects as go
import numpy as np

# Function to load and process data
def calculate_speedup_factors():
    algorithms = ["BFV", "CKKS", "PAILLIER"]
    core_values = [4, 8, 16, 32]
    results = []

    path = "/content/drive/MyDrive/praxisfiles/Benchmark"

    for algorithm in algorithms:
        try:
            # Load cached and uncached data
            cached_file = f"{path}/{algorithm.lower()}_experiment_cached.csv"
            uncached_file = f"{path}/{algorithm.lower()}_experiment_uncached.csv"

            cached_df = pd.read_csv(cached_file)
            uncached_df = pd.read_csv(uncached_file)

            # Process each core value
            for core in core_values:
                # Filter data for this core count
                cached_data = cached_df[cached_df["cores"] == core]
                uncached_data = uncached_df[uncached_df["cores"] == core]

                # Skip if no data for this core count
                if len(cached_data) == 0 or len(uncached_data) == 0:
                    continue

                # Calculate speedup for encryption
                cached_enc = cached_data["avg_add"].mean()
                uncached_enc = uncached_data["avg_add"].mean()
                speedup = uncached_enc / cached_enc

                # Store the result
                results.append({
                    "algorithm": algorithm,
                    "cores": core,
                    "speedup": speedup
                })

        except Exception as e:
            print(f"Error processing {algorithm}: {e}")

    return pd.DataFrame(results)

# Calculate speedup factors
speedup_df = calculate_speedup_factors()

# Create the plot
fig = go.Figure()

# Add bars for each algorithm
for algorithm in speedup_df["algorithm"].unique():
    algo_data = speedup_df[speedup_df["algorithm"] == algorithm]

    fig.add_trace(go.Bar(
        x=algo_data["cores"],
        y=algo_data["speedup"],
        name=algorithm,
        text=[f"{val:.1f}x" for val in algo_data["speedup"]],
        textposition="outside"
    ))

# Update layout
fig.update_layout(
    title="Speedup Factor (Uncached/Cached) for Homomorphic Addition",
    xaxis=dict(
        title="CPU Cores",
        type="category",
        tickvals=[4, 8, 16, 32],
        ticktext=["4", "8", "16", "32"]
    ),
    yaxis=dict(
        title="Speedup Factor (Uncached/Cached) Homomorphic Addition"
    ),
    barmode="group",
    legend=dict(
        title="Algorithm",
        orientation="h",
        yanchor="bottom",
        y=1.02,
        xanchor="right",
        x=1
    )
)

# Ensure y-axis starts at 0
fig.update_yaxes(range=[0, speedup_df["speedup"].max() * 1.2])

# Display the plot
fig.show()

In [ ]:
import pandas as pd
import plotly.express as px

# — load & prepare —
path = "/content/drive/MyDrive/praxisfiles/Benchmark"
bfv       = pd.read_csv(f"{path}/bfv_experiment_uncached.csv").assign(scheme="BFV")
bfv.rename(columns={"poly_modulus_degree":"key_size"}, inplace=True)

paillier = pd.read_csv(f"{path}/paillier_experiment_uncached.csv").assign(scheme="Paillier")
# paillier.rename(columns={"poly_modulus_degree":"key_size"}, inplace=True)  # if needed

ckks      = pd.read_csv(f"{path}/ckks_experiment_uncached.csv").assign(scheme="CKKS")
ckks.rename(columns={"poly_modulus_degree":"key_size"}, inplace=True)

df = pd.concat([bfv, paillier, ckks], ignore_index=True)

# — Encryption Time vs CPU Cores —
fig_enc = px.line(
    df,
    x="cores",
    y="avg_enc",
    color="key_size",
    facet_col="scheme",
    markers=True,
    labels={
        "cores":"CPU Cores",
        "avg_enc":"Encryption Time (s)",
        "key_size":"Key Size (bits)"
    },
    title="Uncached Encryption Time vs. CPU Cores"
)
fig_enc.update_xaxes(type="category")
fig_enc.update_traces(line=dict(shape="linear"))
fig_enc.show()

# — Addition Time vs CPU Cores —
fig_add = px.line(
    df,
    x="cores",
    y="avg_add",
    color="key_size",
    facet_col="scheme",
    markers=True,
    labels={
        "cores":"CPU Cores",
        "avg_add":"Addition Time (s)",
        "key_size":"Key Size (bits)"
    },
    title="Uncached Homomorphic Addition Time vs. CPU Cores"
)
fig_add.update_xaxes(type="category")
fig_add.update_traces(line=dict(shape="linear"))
fig_add.show()

In [ ]:
import pandas as pd
import plotly.express as px

# — load & prepare —
path = "/content/drive/MyDrive/praxisfiles/Benchmark"
bfv       = pd.read_csv(f"{path}/bfv_experiment_cached.csv").assign(scheme="BFV")
bfv.rename(columns={"poly_modulus_degree":"key_size"}, inplace=True)

paillier = pd.read_csv(f"{path}/paillier_experiment_cached.csv").assign(scheme="Paillier")
# paillier.rename(columns={"poly_modulus_degree":"key_size"}, inplace=True)  # if needed

ckks      = pd.read_csv(f"{path}/ckks_experiment_cached.csv").assign(scheme="CKKS")
ckks.rename(columns={"poly_modulus_degree":"key_size"}, inplace=True)

df = pd.concat([bfv, paillier, ckks], ignore_index=True)

# — Encryption Time vs CPU Cores —
fig_enc = px.line(
    df,
    x="cores",
    y="avg_enc",
    color="key_size",
    facet_col="scheme",
    markers=True,
    labels={
        "cores":"CPU Cores",
        "avg_enc":"Encryption Time (s)",
        "key_size":"Key Size (bits)"
    },
    title="Cached Encryption Time vs. CPU Cores"
)
fig_enc.update_xaxes(type="category")
fig_enc.update_traces(line=dict(shape="linear"))
fig_enc.show()

# — Addition Time vs CPU Cores —
fig_add = px.line(
    df,
    x="cores",
    y="avg_add",
    color="key_size",
    facet_col="scheme",
    markers=True,
    labels={
        "cores":"CPU Cores",
        "avg_add":"Addition Time (s)",
        "key_size":"Key Size (bits)"
    },
    title="Cached Homomorphic Addition Time vs. CPU Cores"
)
fig_add.update_xaxes(type="category")
fig_add.update_traces(line=dict(shape="linear"))
fig_add.show()
